#1.ПРЕДОБРАБОТКА ДАННЫХ

Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import numpy as np


Загружаем данные

In [37]:
logs = pd.read_csv('', sep=';')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2341273275.py, line 1)

Проводим предварительный анализ данных

In [ ]:
logs.head()

In [ ]:
logs.describe().T

In [ ]:
logs.info()

In [ ]:
logs.isna().sum()

Наблюдаем колонку без данных - удаляем.

In [ ]:
logs.drop(columns=['Unnamed: 17'], inplace=True)

In [ ]:
logs.duplicated().sum()

дубликатов нет

Удаляем поля с датами поскольку необходимая метрика в числовом формате находится в поле 'time_diff_reg_date'


In [31]:
logs.drop(columns= ['date_purchase', 'date_reg'], inplace=True)

NameError: name 'logs' is not defined

In [ ]:
logs.dtypes

In [ ]:
logs['country_code'] = logs['country_code'].astype(str)
logs.drop(columns=['subplatform_id'], inplace=True)

Из соображений, что платящиго/не платящиго игрока можно лучше классифицировать по признаку откуда он родом, нежели с какого девайса он играет. Удалим колонку "subplatform_id", а колонку "country_code" закодироем с помощью функции pd.get_dummies()

In [ ]:
logs = pd.get_dummies(logs, drop_first=True, dummy_na=False)

In [ ]:
# plt.figure(figsize=(40, 40))
# sns.heatmap(logs.corr(), annot = True, fmt='.2f');

Для того чтоб убрать мультиколлиниарность будем испольвать L2 ругуляризацию

##2.Построение предсказательной модели

Разбиваем данные на обучающею и тестовые выборки

In [ ]:
# from sklearn.model_selection import train_test_split
# X = logs.drop(columns=['category'])
# y = logs.category
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Строим модель классификации логистической регрессии

Оцениваем качество модели

In [32]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# # стандартизировать и привести все признаки к одному масштабу
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train_standard = scaler.transform(X_train)
# X_test_standard = scaler.transform(X_test)

In [33]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score

# lr = LogisticRegression()

# lr.fit(X_train_standard, y_train)
# y_pred = lr.predict(X_test_standard)
# accuracy_lr = accuracy_score(y_test, y_pred)
# precision_lr= precision_score(y_test, y_pred)
# recall_lr = recall_score(y_test, y_pred)

# from sklearn.metrics import classification_report
# print(classification_report(y_test, y_pred))


In [34]:
# #Делаем модель дерево решений

# from sklearn.model_selection import train_test_split
# X_dt = logs.drop(columns=['category'])
# y_dt = logs.category
# X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(X_dt, y_dt, test_size=0.2, random_state=42)

# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# # стандартизировать и привести все признаки к одному масштабу
# scaler_dt = StandardScaler()
# scaler.fit(X_train_dt)
# X_train_standard_dt = scaler.transform(X_train_dt)
# X_test_standard_dt = scaler.transform(X_test_dt)

# from sklearn.tree import DecisionTreeClassifier

# dt = DecisionTreeClassifier()
# dt.fit(X_train_dt, y_train_dt)
# y_pred_dt = dt.predict(X_test_standard_dt)
# accuracy_dt = accuracy_score(y_test, y_pred_dt)
# precision_dt = precision_score(y_test_dt, y_pred_dt)
# recall_dt = recall_score(y_test_dt, y_pred_dt)

# from sklearn.metrics import classification_report
# print(classification_report(y_test_dt, y_pred_dt))

Как видим логистическая регрессия немного дала необходимый результат, но он не достаточен, дерево решений вовсе не справилась с поставленной задачей 

In [35]:
# импортируем необходимые библиотеки
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = logs.drop(columns=['category'], axis=1)
y = logs.category
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


smote = SMOTE(sampling_strategy='minority')
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

lr = LogisticRegression()
lr.fit(X_train_resampled, y_train_resampled)

y_pred = lr.predict(X_test)


print(classification_report(y_test, y_pred))

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
